In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd() #get current working directory

'/content'

In [ ]:
!ls

drive  sample_data


In [ ]:
os.chdir('/content/drive/My Drive')

In [ ]:
!ls


churn_modelling_ANN.ipynb  Churn_Modelling.csv


In [ ]:
import pandas as pd
dataset = pd.read_csv("Churn_Modelling.csv")

In [ ]:
dataset.shape

(10000, 14)

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
features = dataset.iloc[:,3:13].values
labels = dataset.iloc[:,13].values

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

cTransformer = ColumnTransformer([('encoder',OneHotEncoder(),[1,2])], remainder = "passthrough")

features = np.array(cTransformer.fit_transform(features), dtype=np.float32)

features = features[:,1:] # drop the first dummy column for geography.
features = features[:,[0,1,3,4,5,6,7,8,9,10,11]] # drop the column for gender

In [ ]:
features.shape

(10000, 11)

In [ ]:
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test = train_test_split(features,labels,test_size=0.2,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features_train = sc.fit_transform(features_train)
features_test = sc.fit_transform(features_test)

In [ ]:
import keras
from keras.models import Sequential #Keras provides 2 ways of creating the model, Sequential API, Functional API
from keras.layers import Dense #we use dense layers in hidden layer, where each neuron gets inputs from all the neurons in the previous layer.

In [ ]:
classifier = Sequential()

In [ ]:
# adding the first hidden layer
# adding first hidden layer will do two jobs, It adds input layer and one hidden layer

# units = 6 --> 6 nodes in the hidden layer. Number of nodes should meet the max limit criteria. i.e., 2/3rd of input nodes....
# kernel_initializer = 'uniform' --> kernel refers to weights. Assign random uniform weights to all the nodes initiallly
# input_dim = 11 --> add 11 nodes in the input layer as we have 11 features.
# activation ='relu' --> add relu as activation function in hidden layer. Input layer will not have activation function.

classifier.add(Dense(units = 6 , kernel_initializer= 'uniform', activation='relu', input_dim = 11))

In [ ]:
# Adding second hidden layer
classifier.add(Dense(units=6, kernel_initializer= 'uniform', activation = 'relu'))

# Adding the output layer
# Since this is binary classification we use only one node, units = 1
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
# Optimizer will find the optimal parameters that will minimize the loss/ cost function
# adam - adaptive moment estimation is one of the optimizers
# loss  is the cost function
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# batch size =10, total 8000 records will be sent in batches of size 10. so to send all the records, we need 8000/10 = 800 iterations.
# when all the data is passed, it is called one epoch. 
classifier.fit(features_train, labels_train, batch_size = 10 , epochs = 10)

Epoch 1/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4089 - accuracy: 0.8346
Epoch 2/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4088 - accuracy: 0.8326
Epoch 3/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4077 - accuracy: 0.8338
Epoch 4/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4078 - accuracy: 0.8336
Epoch 5/10
800/800 [==============================] - 1s 2ms/step - loss: 0.4074 - accuracy: 0.8340
Epoch 6/10
800/800 [==============================] - 2s 2ms/step - loss: 0.4071 - accuracy: 0.8334
Epoch 7/10
800/800 [==============================] - 2s 2ms/step - loss: 0.4066 - accuracy: 0.8344
Epoch 8/10
800/800 [==============================] - 2s 2ms/step - loss: 0.4064 - accuracy: 0.8340
Epoch 9/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4059 - accuracy: 0.8346
Epoch 10/10
800/800 [==============================] - 1s 1ms/step - loss: 0.4058 - accuracy: 0.8359

In [ ]:
# predict method gives the probability values in the range 0 to 1. we need to use the threshold to get the class predictions.
# Since this is binary classification, we have used sigmoid function, so to get the predictions, we can use
# labels_pred > 0.5 as filter condition, OR
# np.where(labels_pred > threshold, 1,0)

# If we have multi class classification, then we use softmax function, which gives probability values for all the categories. we need to select the category which has maximum probability
# np.argmax(predictions) --> argmax returns the index for which the value is maximum. index will have category names, values will have probabilities
labels_pred = classifier.predict(features_test)
print(labels_pred)
labels_pred = (labels_pred > 0.5)

[[0.26218474]
 [0.27998537]
 [0.15526262]
 ...
 [0.16987422]
 [0.15472174]
 [0.1150555 ]]


In [ ]:
labels_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [ ]:
labels_pred.ndim

2

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test,labels_pred)

In [ ]:
cm

array([[1558,   37],
       [ 292,  113]])

In [ ]:
list(zip(labels_test,labels_pred))